In [ ]:
from runLSTM import buildRunLSTM
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
import pickle
import pandas as pd
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv1D,MaxPooling1D,UpSampling1D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from keras.utils import to_categorical

In [ ]:
with open("data/traindata_01.pkl", "rb") as input_file:
    generated_data = pickle.load(input_file)

list_tuples = [x for x in generated_data]

In [ ]:
b = buildRunLSTM()
X, y, max_features, maxlen, labels = b.read_data(list_tuples)

In [ ]:
scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X)
#test_data = scaler.transform(X_malicious)

In [ ]:
train_data, test_data, y_train, y_test, _, label_test = train_test_split(X, y, labels, test_size=0.4)

In [ ]:
print("Training set (images) shape: {shape}".format(shape=train_data.shape))
print("Testing set (images) shape: {shape}".format(shape=test_data.shape))

In [ ]:
label_dict = {
 0: 'Normal',
 1: 'DGA',
}

In [ ]:
train_data.dtype, test_data.dtype

In [ ]:
np.max(train_data), np.max(test_data)

In [ ]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_ground,valid_ground = train_test_split(train_data,
                                                             train_data,
                                                             test_size=0.2,
                                                             random_state=13)

In [ ]:
train_X = np.reshape(train_X, (len(train_X), maxlen, 1))
valid_X = np.reshape(valid_X, (len(valid_X), maxlen, 1))
train_ground = np.reshape(train_ground, (len(train_ground), maxlen, 1))
valid_ground = np.reshape(valid_ground, (len(valid_ground), maxlen, 1))


In [ ]:
batch_size = 64
epochs = 50
inChannel = 1
x, y = 1, 59
input_img = Input(shape = (59, 1))
num_classes = 2

In [ ]:
def encoder(input_img):
    #encoder
    #input = 1 X 59 (wide and thin)
    conv1 = Conv1D(3, 3, activation='relu', padding='same')(input_img) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(3, 3, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling1D(pool_size=1)(conv1) #14 x 14 x 32
    conv2 = Conv1D(6, 3, activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(6, 3, activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    #conv2 = MaxPooling1D(pool_size=1)(conv2) #7 x 7 x 64
    #conv3 = Conv1D(9, 3, activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    #conv3 = BatchNormalization()(conv3)
    #conv3 = Conv1D(9, 3, activation='relu', padding='same')(conv3)
    #conv3 = BatchNormalization()(conv3)
    #conv4 = Conv1D(12, 3, activation='relu', padding='same')(conv3) #7 x 7 x 256 (small and thick)
    #conv4 = BatchNormalization()(conv4)
    #conv4 = Conv1D(12, 3, activation='relu', padding='same')(conv4)
    #conv4 = BatchNormalization()(conv4)
    return conv2

def decoder(conv2):    
    #decoder
    #conv5 = Conv1D(9, 3, activation='relu', padding='same')(conv4) #7 x 7 x 128
    #conv5 = BatchNormalization()(conv5)
    #conv5 = Conv1D(9, 3, activation='relu', padding='same')(conv5)
    #conv5 = BatchNormalization()(conv5)
    conv6 = Conv1D(6, 3, activation='relu', padding='same')(conv2) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv1D(6, 3, activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    #up1 = UpSampling1D(2)(conv6) #14 x 14 x 64
    conv7 = Conv1D(3, 3, activation='relu', padding='same')(conv6) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv1D(3, 3, activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    #up2 = UpSampling1D(2)(conv7) # 28 x 28 x 32
    decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(conv7) # 28 x 28 x 1
    return decoded

In [ ]:
autoencoder = Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())
autoencoder.summary()

In [ ]:
autoencoder_train = autoencoder.fit(train_X[0:10000], train_ground[0:10000], batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X[0:1000], valid_ground[0:1000]))

In [ ]:
train_X.shape